In [1]:
from retrieval import Pyserini_Search
from reader import Reader
# import os
# os.environ["CUDA_VISIBLE_DEVICES"]="5"  # specify which GPU(s) to be used


In [2]:
retriever = Pyserini_Search(index_path='indexes/full_wiki', device='cuda:5')
reader_model = Reader(model_checkpoint='nguyenvulebinh/vi-mrc-base', device=5)

Some weights of the model checkpoint at nguyenvulebinh/vi-mrc-base were not used when initializing RobertaModel: ['qa_outputs.weight', 'qa_outputs.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [3]:
def get_answer(question):
    contexts, ranking_scores = retriever.search(question, 100, use_rerank=True)
    print(ranking_scores)
    result, all_answers = reader_model(question, contexts, ranking_scores)
    return result, all_answers

In [4]:
import time
start = time.time()
result, all_answers = get_answer('Cộng hòa Weimar chính thức thay thế đế quốc Đức kể từ sau sự kiện nào?')
print(time.time()-start)

[9.9727672e-01 2.3931810e-03 2.3495116e-04 1.9608128e-04 1.5035507e-04
 1.0542595e-04 9.2069189e-05 7.2127463e-05 5.2891464e-05 4.9898445e-05]
1.524564266204834


In [6]:
all_answers_lst = []
for ans in all_answers:
    all_answers_lst.append(
        { "answer": ans.text,
            "score": ans.score,
            'context': ans.context
            }
    )
top5 = sorted(all_answers_lst, key=lambda k: k['score'], reverse=True)[:5]
for i in top5:
    print("<Passage:{},\n score:{}>".format(i['context'], i['score']))

<Passage:năm 1871 , đức trở_thành một quốc_gia dân_tộc khi hầu_hết các quốc_gia đức thống_nhất trong đế_quốc đức do phổ chi_phối . sau chiến_tranh thế_giới thứ nhất và cách_mạng đức 1918 - 1919 , đế_quốc này bị thay_thế bằng cộng hòa weimar theo chế_độ nghị_viện . chế_độ độc_tài quốc xã được hình_thành vào năm 1933 , dẫn tới chiến_tranh thế_giới thứ hai và một nạn diệt chủng . sau một giai_đoạn đồng_minh chiếm_đóng , hai nước đức được thành_lập : cộng_hòa liên_bang đức và cộng hòa dân_chủ đức . năm 1990 , quốc_gia được tái thống_nhất .,
 score:0.9289566873298654>
<Passage:trong chiến_tranh thế_giới thứ hai , đế_quốc nhật bản đảo_chính pháp và chiếm toàn_thể đông dương . với sự chuẩn_bị từ trước đó trong nhiều năm và với sự ủng_hộ của người dân , ngay sau khi hay tin đế_quốc nhật đầu_hàng quân đồng_minh , hồ chí minh đã lãnh_đạo việt minh giành lại chính_quyền từ tay nhật . ngày 2 tháng 9 năm 1945 , hồ chí minh đọc tuyên_ngôn độc_lập và thành_lập nước việt nam dân_chủ cộng_hòa , nhà_nướ

: 

## Evaluation

In [6]:
from base import Question
from tqdm.notebook import trange, tqdm
import json

In [7]:
def extract_squad_questions(squad_filename):
    data = json.load(open(squad_filename, 'r'))
    data = data["data"]
    questions = []
    for article in data:
        for paragraph in article["paragraphs"]:
            gt_context = paragraph['context']
            for qa in paragraph["qas"]:
                id_ = qa["id"]
                question = qa["question"]
                
                questions.append(Question(question, id_, gt_context))
    return questions

In [10]:
questions = extract_squad_questions('./ViQuAD/test_ViQuAD.json')
all_answer = []
for question in tqdm(questions):
    answers, final_answers = get_answer(question.text)
    # answers, final_answers = reader_model(question.text, [question.gt_context])   #Evaluate Reader model
    final_answers_lst = []
    for ans in final_answers:
        final_answers_lst.append(
            {"id": question.id,
                "answer": ans.text,
                "phrase_score": ans.score,
                "paragraph_score": ans.ctx_score,
                }
        )
    all_answer.append(final_answers_lst)


  0%|          | 0/2210 [00:00<?, ?it/s]

/opt/conda/lib/python3.8/site-packages/transformers/pipelines/base.py:1070: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/transformers/pipelines/base.py:1070: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/transformers/pipelines/base.py:1070: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/transformers/pipelines/base.py:1070: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
/opt/conda/lib/python3.8/site-packages/transformers/pipelines/base.py:1070: UserWarning: You seem to be using the pipelines sequentially on 

In [ ]:
json.dump(all_answer, open('predict.json', 'w'), indent=4)

In [11]:
all_answer[0]

[{'id': 'uit_01__00061_0_1',
  'answer': 'chiến tranh thế giới thứ nhất và cách mạng đức 1918 - 1919',
  'phrase_score': 0.9289511670362849,
  'paragraph_score': 0.9972766},
 {'id': 'uit_01__00061_0_1',
  'answer': 'bầu cử liên bang vào năm 1930',
  'phrase_score': 0.0009768193811147963,
  'paragraph_score': 0.0012132793},
 {'id': 'uit_01__00061_0_1',
  'answer': 'hiệp ước lausanne ngày 24 tháng 7 năm 1923',
  'phrase_score': 3.955433109109575e-05,
  'paragraph_score': 0.00023492383},
 {'id': 'uit_01__00061_0_1',
  'answer': 'chiến tranh thế giới thứ hai',
  'phrase_score': 5.052557231102913e-05,
  'paragraph_score': 0.0002245679},
 {'id': 'uit_01__00061_0_1',
  'answer': 'sự sụp đổ của bức tường berlin ngày 9 tháng 11 năm 1989',
  'phrase_score': 5.026794544876062e-05,
  'paragraph_score': 0.00013229303},
 {'id': 'uit_01__00061_0_1',
  'answer': 'nhật bản xâm nhập đông bắc trung quốc',
  'phrase_score': 2.8057736012353446e-05,
  'paragraph_score': 0.000105372375},
 {'id': 'uit_01__000

In [ ]:
answers = {}

predictions = json.load(open('predict.json', 'r'))
for predict_id, predict in enumerate(predictions):
    id_ = predict[0]['id']

    for i in predict:
        i['total_score'] = i['phrase_score'] #* i['paragraph_score']
    best_answer = sorted(predict, key=lambda k: k['total_score'], reverse=True)[0]
    answers[id_] = best_answer['answer']

json.dump(answers, open("tmp.answer", 'w'))

In [ ]:
from eval import squad_v1_eval
result = squad_v1_eval( './ViQuAD/test_ViQuAD.json', 'tmp.answer')

total: 2210


In [ ]:
result

{'exact_match': 52.66968325791855,
 'f1': 60.25669472081291,
 'recall': 59.97796853105979,
 'precision': 67.53744671617973,
 'cover': 54.25339366515837,
 'overlap': 80.63348416289593}

## Reader Evalute

In [ ]:
'''
XLM_base
{'exact_match': 61.040723981900456,
 'f1': 82.43619252485041,
 'recall': 80.71308856244376,
 'precision': 94.30660352719175,
 'cover': 67.73755656108597,
 'overlap': 98.23529411764706
 }

 '''

'''
XLM_Large
 {'exact_match': 69.90950226244344,
 'f1': 76.98466658573287,
 'recall': 75.76102341755725,
 'precision': 98.26953853085077,
 'cover': 72.17194570135747,
 'overlap': 99.14027149321267}
'''

E2E Evaluate

In [13]:
print('Giai \u0111o\u1ea1n n\u0103m 1955-1976, Ph\u1ea1m V\u0103n \u0110\u1ed3ng n\u1eafm gi\u1eef ch\u1ee9c v\u1ee5 g\u00ec')

Giai đoạn năm 1955-1976, Phạm Văn Đồng nắm giữ chức vụ gì


In [ ]:
'''
E2E: BertLarge
{'exact_match': 52.66968325791855,
 'f1': 60.25669472081291,
 'recall': 59.97796853105979,
 'precision': 67.53744671617973,
 'cover': 54.25339366515837,
 'overlap': 80.63348416289593}
'''
Base:
{'exact_match': 35.74660633484163,
 'f1': 45.53762041245419,
 'recall': 45.52465145440133,
 'precision': 49.04903404790284,
 'cover': 38.19004524886878,
 'overlap': 68.82352941176471}


'''